In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from src import supabase_client
from src.update_pipeline import update_training_week, _update_training_week
from src.types.update_pipeline import ExeType
from src.utils import get_last_sunday
from src import auth_manager
from src.activities import get_weekly_summaries, get_daily_activity
from src.auth_manager import get_strava_client
from src.llm import get_completion_json
from src.constants import COACH_ROLE
from src.utils import datetime_now_est

from enum import StrEnum
from freezegun import freeze_time
import os
import numpy as np
from pydantic import BaseModel, Field
from datetime import timedelta
import matplotlib.pyplot as plt

plt.style.use("seaborn-v0_8")

# Update Jamie's Training Plan

In [ ]:
user = supabase_client.get_user(os.environ["JAMIES_ATHLETE_ID"])
assert user.preferences.race_date is not None
assert user.preferences.race_distance is not None

@freeze_time(f"{get_last_sunday()} 18:00:00")
def frozen_update_training_week_new_week():
    return _update_training_week(user, ExeType.NEW_WEEK)

response = frozen_update_training_week_new_week()

# New Week Update Training Week with Side Effects

In [ ]:
@freeze_time(f"{get_last_sunday()} 18:00:00")
def frozen_update_training_week_new_week(user):
    return update_training_week(user, ExeType.NEW_WEEK)

for user in supabase_client.list_users(debug=True):
    auth_manager.authenticate_athlete(user.athlete_id)
    try:
        response = frozen_update_training_week_new_week(user)
        print(response)
    except Exception as e:
        print(e)

# Mid Week Update Training Week with Side Effects

In [ ]:
user = supabase_client.get_user(os.environ["JAMIES_ATHLETE_ID"])
auth_manager.authenticate_athlete(user.athlete_id)
response = update_training_week(user, ExeType.MID_WEEK)
print(response)